In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jupyros
import rospy
import rospkg
from turtlesim.srv import Spawn
from random import randint
from math import radians
from ipywidgets import Image
from ipycanvas import hold_canvas, Canvas, MultiCanvas
from time import time
from turtlesim.msg import Pose

In [3]:
rospy.init_node("superturtle")

# Turtlesim

In [4]:
# !rosrun turtlesim turtlesim_node
# !rosrun turtlesim turtle_teleop_key
# jupyros.client('spawn', Spawn)

In [5]:
def get_random_turtle():
    turtles_svg = ['hydro', 'indigo', 'kinetic', 'lunar']
    turtles_png = ['box-turtle', 'diamondback', 'electric', 'fuerte', 'groovy', 'hydro', 'indigo', 'jade',
                   'kinetic', 'lunar', 'melodic', 'noetic', 'robot-turtle', 'sea-turtle', 'turtle']
    
    # Find path to a random turtle image
    r = rospkg.RosPack()
    turtlesim_path = r.get_path('turtlesim')
    turtle_path = turtlesim_path + '/images/' + turtles_png[randint(0, len(turtles_png)-1)] + '.png'

    turtle_img = Image.from_file(turtle_path)

    return turtle_img


def spawn_turtle():
    spawn_canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
    
    # Water
    spawn_canvas.fill_style = "#4556FF"
    spawn_canvas.fill_rect(0, 0, spawn_canvas.width, spawn_canvas.height)
    
    # Turtle
    turtle_img = get_random_turtle()   
    turtle_size = 100  
    turtle_canvas = Canvas(width=turtle_size, height=turtle_size)  
    turtle_canvas.draw_image(turtle_img, width=turtle_size)

    
    # Center of turtle and canvas from top left
    x = (spawn_canvas.width - turtle_size) / 2
    y = (spawn_canvas.height - turtle_size) / 2
    
    # Water canvas
    spawn_canvas.translate(x, y)
    spawn_canvas.rotate(radians(90))
    spawn_canvas.draw_image(turtle_img, width=turtle_size)
    spawn_canvas.rotate(radians(-90))
    spawn_canvas.translate(-x, -y)

    return turtle_canvas, spawn_canvas

In [41]:
## WITH MULTI CANVAS

def spawn_turtle_multi():
    # [0] Background
    # [1] Path line
    # [2] Turtle
    # [3] Ghost turtle
    spawn_canvas = MultiCanvas(4, width=1600, height=1200, layout=dict(width="100%"))
    
    # Water background
    spawn_canvas[0].fill_style = "#4556FF"
    spawn_canvas[0].fill_rect(0, 0, spawn_canvas.width, spawn_canvas.height)
    
    # Turtle
    turtle_img = get_random_turtle()   
    turtle_size = 100  
    turtle_canvas = Canvas(width=turtle_size, height=turtle_size)  
    turtle_canvas.draw_image(turtle_img, width=turtle_size)
    
    # Center of turtle and canvas from top left
    x = (spawn_canvas[0].width - turtle_size) / 2
    y = (spawn_canvas[0].height - turtle_size) / 2
    
    # Turtle canvas
    with hold_canvas(spawn_canvas):
        spawn_canvas[2].translate(x, y)
        spawn_canvas[2].rotate(radians(90))
        spawn_canvas[2].draw_image(turtle_img, width=turtle_size)
        spawn_canvas[2].rotate(radians(-90))
        spawn_canvas[2].translate(-x, -y)

    return turtle_canvas, spawn_canvas

### Translate and rotate turtle

In [7]:
def move_turtle(turtle_canvas, water_canvas, x, y, theta, turtle_size=100): 
    with hold_canvas(water_canvas):
        movement = "Moving to {}, {}, with {} radians"
#         print(movement.format(x,y,theta))
        
        # Clear old animation step
        water_canvas.clear()

        # New animation drawings
        water_canvas.fill_style = "#4556FF"
        water_canvas.fill_rect(0, 0, water_canvas.width, water_canvas.height)
        
                
        # Offsets for turtle center and orientation
        x_offset = -turtle_size / 2
        y_offset = -turtle_size / 2
        theta_offset = theta - radians(90)
        
        # Transform canvas
        water_canvas.translate(x, y)
        water_canvas.rotate(-theta_offset)

        water_canvas.draw_image(turtle_canvas, 
                                x=x_offset, 
                                y=y_offset, 
                                width=turtle_size)
        
        # Revert transformation
        water_canvas.rotate(theta_offset)
        water_canvas.translate(-x, -y)
        
    return water_canvas

In [57]:
def move_turtle_multi(turtle_canvas, canvas, x, y, theta, turtle_size=100): 
    with hold_canvas(canvas):
        movement = "Moving to {}, {}, with {} radians"
#         global counter
#         if time() - counter > 3:
#             print(movement.format(x,y,theta))

        # Offsets for turtle center and orientation
        x_offset = -turtle_size / 2
        y_offset = -turtle_size / 2
        theta_offset = theta - radians(90)

        def draw_turtle(n):
            canvas[n].translate(x,y)
            canvas[n].rotate(-theta_offset)
            canvas[n].draw_image(turtle_canvas, 
                                 x=x_offset,
                                 y=y_offset,
                                 width=turtle_size)
            canvas[n].rotate(theta_offset)
            canvas[n].translate(-x,-y)   
            return canvas

        # Draw turtle in new position on foremost canvas
        draw_turtle(3)

        # Clear old turtle
        canvas[2].clear()

        # Draw new turtle again on middle canvas
        draw_turtle(2)

        # Clear foremost canvas
        canvas[3].clear()
        
        # Draw line marker
        canvas[1].fill_style = "#B3B8FF"
        canvas[1].fill_circle(x, y, radius=3)
        
        
    return canvas

In [9]:
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * water[0].width
    y_convert = (convert_factor - y) / convert_factor * water[0].height
    
    return x_convert, y_convert

In [10]:
# from time import sleep

# x = water_canvas.width // 2
# y = water_canvas.height // 2

# for i in range(360):
#     move_turtle(turtle_canvas, water_canvas, x+i, y+i, theta=radians(i))
#     sleep(0.01)

### Publisher

In [11]:
topic_name = '/turtle1/pose'

# jupyros.publish(topic_name, Pose)

### Animation

In [ ]:
turtle_canvas, water_canvas = spawn_turtle()
display(water_canvas)

### Subscriber

In [ ]:
elapsed_time = 0 # seconds
start_time = time()

def callback_move_turtle(msg):
    global start_time
    
    elapsed_time = time() - start_time

    if elapsed_time > 0.1:
        start_time = time()
        x_canvas, y_canvas = convert_xy_units(msg.x, msg.y)
        move_turtle(turtle_canvas, water_canvas, x_canvas, y_canvas, msg.theta)

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtle(msg))

### Multi Canvas

In [59]:
turtle, water = spawn_turtle_multi()
display(water)

counter = time()
elapsed_time = 0 # seconds
start_time = time()

MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [60]:
def callback_move_turtle(msg):
    global start_time
    
    elapsed_time = time() - start_time

    if elapsed_time > 0.05:
        start_time = time()
        x_canvas, y_canvas = convert_xy_units(msg.x, msg.y)
        move_turtle_multi(turtle, water, x_canvas, y_canvas, msg.theta)

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtle(msg))

Removing previous callback, only one redirection possible right now


In [ ]:
import rostopic

pubs, _ = rostopic.get_topic_list()
pubs

# Retrieve name of last turtle spawned
last_turtle = pubs[-1][0].split('/')[1]
print(last_turtle)